In [1]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:

def chamar_arquivo():
    current_dir = os.getcwd()

    # Junta o caminho para o arquivo 'arquivo.csv' (que está no diretório acima)
    file_path = os.path.join(current_dir, '..', 'teste.csv')

    # Resolve o caminho absoluto
    file_path = os.path.abspath(file_path)

    # Agora você pode ler o arquivo
    df = pd.read_csv(file_path)
    return df


def tirar_outliers(df):
    Q1 = df['preço'].quantile(0.25)
    Q3 = df['preço'].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 2 * IQR
    limite_superior = Q3 + 2 * IQR
    df = df[(df['preço'] >= limite_inferior) & (df['preço'] <= limite_superior)]
    return df


def separar_dados(df):
    numericas = [col for col in df.columns if df[col].dtype in ['float64', 'int64'] and col not in ['preço', 'preco p/ m²']]
    X = df[numericas]
    y = df['preço']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

    

In [6]:
def load_and_train_model():
    df = chamar_arquivo()
    
    df = df.drop_duplicates()
    df = df[df['bairro'] != 'Siqueira']
    df = df.dropna(subset=['preço'])
    df = df[df['condominio'] < 10000]
    df.drop(columns=['Unnamed: 0','Unnamed: 0.1','IDH_x','IDH-Renda_x', 'IDH-Longevidade_x',
                       'IDH-Educação_x', 'Regional_x','Regional_y','numero','Regional','preco p/ m²',
                       'IDH-Educação','IDH','preço_cond_ratio'], inplace=True)
    numericas = [col for col in df.columns if df[col].dtype in ['float64', 'int64'] and col not in ['preço', 'preco p/ m²']]
    df = tirar_outliers(df)
    X_train, X_test, y_train, y_test = separar_dados(df)

    
    best_params = {
        'colsample_bytree': 1.0, 
        'learning_rate': 0.1, 
        'max_depth': 3, 
        'n_estimators': 500, 
        'subsample': 0.9
    }
    
    preprocessor = ColumnTransformer([('num', StandardScaler(), numericas)])
    
    xgb_pipeline = make_pipeline(
        preprocessor,
        XGBRegressor(**best_params)
    )

    xgb_pipeline.fit(X_train, y_train)
    
    return xgb_pipeline, numericas

if __name__ == '__main__':
    model, numericas = load_and_train_model()
    # Aqui você pode salvar o modelo usando pickle ou joblib, se desejar.


In [7]:
numericas

['area m²',
 'Quartos',
 'banheiros',
 'vagas',
 'condominio',
 'latitude',
 'longitude',
 'quartos_por_m²',
 'banheiros_por_quarto',
 'IDH-Longevidade',
 'IDH-Renda']